In [203]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

nola = pd.read_csv("Nola_all.csv")
nola['game_date'] = pd.to_datetime(nola['game_date'])

In [204]:
#Create index because sv_id was incomplete
nola["pitch_pk"] = None
for ix, d in enumerate(nola.game_date):
    nola.pitch_pk[ix] = d.strftime("%Y%m%d")+str(nola.at_bat_number[ix]).zfill(3)+str(nola.pitch_number[ix]).zfill(2)

nola=nola.sort_values(by="pitch_pk")
nola.reset_index(inplace=True)
nola.drop(columns=['player_name', 'game_date', 'batter_name', 'inning_topbot', 'des', 'at_bat_number'], axis=1, inplace=True)
nola.drop(columns=['index'],  axis=1, inplace=True)

In [205]:
#Dummies made it look like runners are always on
nola.on_3b = np.logical_xor(nola.on_3b,1).astype(int)
nola.on_2b = np.logical_xor(nola.on_2b,1).astype(int)
nola.on_1b = np.logical_xor(nola.on_1b,1).astype(int)

In [206]:
#Now make an interaction variable for balls and strikes. Higher favors pitcher.
nola["count_rating"] = .5 + nola.strikes*(.5/2) - nola.balls*(.5/3)  #3-0 count is a 0, 0-2 count is a 1
nola["runner_pressure"] = (nola.on_1b+2*nola.on_2b+3*nola.on_3b)/6.
nola["score_diff"] = nola.fld_score - nola.bat_score
nola["home_fld"] = (nola.home_team=="PHI").astype(int)

In [207]:
nola["opponent"] = None
for ix, _ in enumerate(nola.opponent):
    if nola.home_fld[ix] == 1:
        nola.opponent[ix] = nola.away_team[ix]
    else:
        nola.opponent[ix] = nola.home_team[ix]

In [208]:
nola["woba_hist"] = None
nola["lsangle_hist"] = None
nola["prev_hth"] = 0

for bid in nola.batter.unique():
    hist = nola[nola.batter==bid][nola.woba_value>=0]
    while hist.shape[0] > 1:
        nola.woba_hist.iloc[hist.index[-1]] = hist[:-1].woba_value.mean()
        nola.lsangle_hist.iloc[hist.index[-1]] = hist[:-1].launch_speed_angle.mean()
        nola.prev_hth.iloc[hist.index[-1]] = hist.shape[0]-1
        hist.drop(hist.index[-1], inplace=True)

In [209]:
x = nola.shape[0]-1
for ix, fill in enumerate(reversed(nola.woba_hist)):
    if ix==x:
        break
    if isinstance(fill, float):
        if nola.batter[x-ix] == nola.batter[x-ix-1]:
            nola.woba_hist[x-ix-1] = fill

In [210]:
for ix, fill in enumerate(reversed(nola.prev_hth)):
    if ix==x:
        break
    if nola.batter[x-ix] == nola.batter[x-ix-1]:
        nola.prev_hth[x-ix-1] = fill

In [211]:
for ix, fill in enumerate(reversed(nola.lsangle_hist)):
    if ix==x:
        break
    if isinstance(fill, float):
        if nola.batter[x-ix] == nola.batter[x-ix-1]:
            nola.lsangle_hist[x-ix-1] = fill

In [187]:
nola[nola.batter==592669]

,home_team,away_team,pitch_type,batter,pitch_number,inning,bat_score,fld_score,stand,balls,...,description,game_pk,pitch_pk,count_rating,runner_pressure,score_diff,home_fld,woba_hist,prev_hth,lsangle_hist
4014,PHI,SD,CU,592669,1,1,0,0,0,0,...,called_strike,491418,2017070800401,0.500000,0.500000,0,1,None,0,None
4015,PHI,SD,FF,592669,2,1,0,0,0,0,...,ball,491418,2017070800402,0.750000,0.500000,0,1,None,0,None
4016,PHI,SD,CH,592669,3,1,0,0,0,1,...,hit_into_play,491418,2017070800403,0.583333,0.500000,0,1,None,0,None
4048,PHI,SD,CH,592669,1,4,0,0,0,0,...,foul,491418,2017070802201,0.500000,0.000000,0,1,0.0,1,1.0
4049,PHI,SD,FF,592669,2,4,0,0,0,0,...,called_strike,491418,2017070802202,0.750000,0.000000,0,1,0.0,1,1.0
4050,PHI,SD,FF,592669,3,4,0,0,0,0,...,called_strike,491418,2017070802203,1.000000,0.000000,0,1,0.0,1,1.0
4081,PHI,SD,FF,592669,1,7,0,1,0,0,...,called_strike,491418,2017070804101,0.500000,0.000000,1,1,0.0,2,1.0
4082,PHI,SD,FF,592669,2,7,0,1,0,0,...,swinging_strike,491418,2017070804102,0.750000,0.000000,1,1,0.0,2,1.0
4083,PHI,SD,CU,592669,3,7,0,1,0,0,...,ball,491418,2017070804103,1.000000,0.000000,1,1,0.0,2,1.0
4084,PHI,SD,CU,592669,4,7,0,1,0,1,...,hit_into_play_no_out,491418,2017070804104,0.833333,0.000000,1,1,0.0,2,1.0


In [212]:
#Create pitch count & batter count.
nola["pitch_count"] = None
nola["batter_count"] = None
x=len(nola.game_pk)-1
i=0
b=1

for ix, pk in enumerate(nola.game_pk):
    i += 1
    
    nola.pitch_count[ix]=i
    nola.batter_count[ix]=b
    
    if ix==x:
        break
    if nola.batter[ix] != nola.batter[ix+1]:
        b += 1
    if pk != nola.game_pk[ix+1]:
        i=0
        b=1

In [213]:
nola.drop(columns=['home_team', 'away_team', 'woba_value', 'launch_speed_angle',
                   'description'],  axis=1, inplace=True)

In [214]:
nola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13170 entries, 0 to 13169
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pitch_type       13170 non-null  object 
 1   batter           13170 non-null  int64  
 2   pitch_number     13170 non-null  int64  
 3   inning           13170 non-null  int64  
 4   bat_score        13170 non-null  int64  
 5   fld_score        13170 non-null  int64  
 6   stand            13170 non-null  int64  
 7   balls            13170 non-null  int64  
 8   strikes          13170 non-null  int64  
 9   outs_when_up     13170 non-null  int64  
 10  on_3b            13170 non-null  int64  
 11  on_2b            13170 non-null  int64  
 12  on_1b            13170 non-null  int64  
 13  game_pk          13170 non-null  int64  
 14  pitch_pk         13170 non-null  object 
 15  count_rating     13170 non-null  float64
 16  runner_pressure  13170 non-null  float64
 17  score_diff  

In [204]:
nola.groupby(by='game_pk').count().batter.sort_values(ascending=False)

game_pk
567208    117
565970    115
530622    114
630853    113
631153    113
         ... 
448253     68
529416     68
415964     67
415638     65
565899     63
Name: batter, Length: 139, dtype: int64

In [205]:
nola.groupby(by='pitch_type').count().inning.sort_values(ascending=False)

pitch_type
CU    4138
FF    4015
SI    2761
CH    2256
Name: batter, dtype: int64

In [58]:
nola.groupby(by='launch_speed_angle').count().inning.sort_values(ascending=False)

launch_speed_angle
2.0    860
4.0    564
3.0    459
5.0    117
6.0    116
1.0     83
Name: inning, dtype: int64

In [59]:
nola.groupby(by='batter').count().inning.sort_values(ascending=False)

batter
543685    182
518692    180
624424    178
455976    169
665742    160
         ... 
591741      2
543194      1
621261      1
621219      1
647336      1
Name: inning, Length: 506, dtype: int64

In [215]:
nola.to_csv('Nola_featured.csv', index=False)